In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    print(text_norm)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

## LJ Speech

In [4]:
! head ../phone_valid.txt


meian/meian_1192.wav|0|sil-j-i-g-a-s-a-k-u-y-u:-sh-i-m-e-q-p-a-n-a-sh-i-n-i-sh-i-t-a-m-a-N-m-a-n-a-n-o-y-o-k-i-q-t-o-i-y-a-n-a-h-i-t-o-k-o:-y-u-q-t-a-o-n-o-b-e-h-a-i-ts-u-m-o-s-u-r-u-k-u-s-e-n-o-t-o:-r-i-sil
meian/meian_2489.wav|0|sil-w-a-g-o:-n-o-t-o-k-i-n-i-n-o-m-e-b-a-s-e-i-sh-i-N-g-a-m-a-s-u-m-a-s-u-k-e-N-z-e-N-n-i-n-a-r-u-s-o:-sh-i-t-e-sh-i-N-t-a-i-h-a-i-y-o-i-y-o-ky-o:-s-o:-n-i-n-a-r-u-sil
meian/meian_1746.wav|0|sil-k-a-n-o-j-o-h-a-k-o-n-o-f-u-j-i-N-o-m-i-r-u-y-a-i-n-a-y-a-u-y-a-u-y-a-sh-i-k-u-a-t-a-m-a-o-s-a-g-e-t-e-m-a-i-d-o-ts-u-d-a-g-a-g-o-y-a-q-k-a-i-n-i-n-a-r-i-m-a-sh-i-t-e-t-o-y-u-q-t-a-sil
meian/meian_5393.wav|0|sil-n-a-N-b-o-a-t-a-sh-i-d-a-q-t-e-a-n-a-t-a-n-o-k-a-N-g-a-e-t-e-i-r-a-q-sh-a-r-u-h-o-d-o-t-a-i-h-e-i-j-a-n-a-i-N-d-a-k-a-r-a-ts-u-d-a-h-a-k-o-t-a-e-n-a-k-a-q-t-a-sh-i-k-a-sh-i-o-n-o-b-e-h-a-y-a-m-e-n-a-k-a-q-t-a-sil
meian/meian_1877.wav|0|sil-y-o-r-i-m-i-ch-i-m-o-s-e-z-u-n-i-t-o:-r-i-k-a-r-a-h-a-N-ch-o:-h-o-d-o:-k-o-n-a-q-t-a-t-o-k-o-r-o-n-i-a-r-u-sh-i-N-r-a-sh-i

In [14]:
ls ../JNAS_K1/*.wav

../JNAS_K1/F001_KAN_NF001002_000.wav  ../JNAS_K1/F003_KAN_NF003088_000.wav
../JNAS_K1/F001_KAN_NF001003_000.wav  ../JNAS_K1/F003_KAN_NF003089_000.wav
../JNAS_K1/F001_KAN_NF001004_000.wav  ../JNAS_K1/F003_KAN_NF003090_000.wav
../JNAS_K1/F001_KAN_NF001005_000.wav  ../JNAS_K1/F003_KAN_NF003091_000.wav
../JNAS_K1/F001_KAN_NF001006_000.wav  ../JNAS_K1/F003_KAN_NF003092_000.wav
../JNAS_K1/F001_KAN_NF001007_000.wav  ../JNAS_K1/F003_KAN_NF003093_000.wav
../JNAS_K1/F001_KAN_NF001008_000.wav  ../JNAS_K1/F003_KAN_NF003094_000.wav
../JNAS_K1/F001_KAN_NF001009_000.wav  ../JNAS_K1/F003_KAN_NF003095_000.wav
../JNAS_K1/F001_KAN_NF001010_000.wav  ../JNAS_K1/F003_KAN_NF003096_000.wav
../JNAS_K1/F001_KAN_NF001011_000.wav  ../JNAS_K1/F003_KAN_NF003097_000.wav
../JNAS_K1/F001_KAN_NF001012_000.wav  ../JNAS_K1/F003_KAN_NF003098_000.wav
../JNAS_K1/F001_KAN_NF001013_000.wav  ../JNAS_K1/F003_KAN_NF003099_000.wav
../JNAS_K1/F001_KAN_NF001014_000.wav  ../JNAS_K1/F003_KAN_NF003100_000.wav
../JNAS_K1/F001_KAN_NF001

In [6]:
#hps = utils.get_hparams_from_file("./css_config.json")
hps = utils.get_hparams_from_file("./logs/mix2/config.json")

In [59]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("logs/mix2/G_1000.pth", net_g, None)

RuntimeError: Error(s) in loading state_dict for SynthesizerTrn:
	size mismatch for enc_p.emb.weight: copying a param with shape torch.Size([45, 192]) from checkpoint, the shape in current model is torch.Size([26, 192]).

In [2]:
txt="sil-w-a-g-o:-n-o-t-o-k-i-n-i-n-o-m-e-b-a-s-e-i-sh-i-N-g-a-m-a-s-u-m-a-s-u-k-e-N-z-e-N-n-i-n-a-r-u-s-o:-sh-i-t-e-sh-i-N-t-a-i-h-a-i-y-o-i-y-o-ky-o:-s-o:-n-i-n-a-r-u-sil"
stn_tst = get_text(txt, hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

NameError: name 'hps' is not defined

## VCTK

In [7]:
#hps = utils.get_hparams_from_file("./configs/vctk_base.json")
hps = utils.get_hparams_from_file("./logs/mix2/config.json")

In [8]:
hps.data.n_speakers,

(2,)

In [9]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("logs/mix2/G_99000.pth", net_g, None)

In [11]:
txt="sil-k-o-m-e-sil"
stn_tst = get_text(txt, hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([1]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

[36, 33, 15, 30, 42, 36]


### Voice Conversion

In [ ]:
dataset = TextAudioSpeakerLoader(hps.data.validation_files, hps.data)
collate_fn = TextAudioSpeakerCollate()
loader = DataLoader(dataset, num_workers=8, shuffle=False,
    batch_size=1, pin_memory=True,
    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x.cuda() for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([1]).cuda()
    sid_tgt2 = torch.LongTensor([2]).cuda()
    sid_tgt3 = torch.LongTensor([4]).cuda()
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))